In [1]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

In [2]:
def get_model_instance_segmentation_resnet50_fpn_v2(num_classes,rpn_s_t=0, box_s_t=0.05):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [3]:
model = get_model_instance_segmentation_resnet50_fpn_v2(num_classes=2)
from torchinfo import summary
summary(model, input_size=(2, 1, 28, 28))


Layer (type:depth-idx)                             Output Shape              Param #
MaskRCNN                                           [100, 4]                  --
├─GeneralizedRCNNTransform: 1-1                    [2, 3, 800, 800]          --
├─BackboneWithFPN: 1-2                             [2, 256, 13, 13]          --
│    └─IntermediateLayerGetter: 2-1                [2, 2048, 25, 25]         --
│    │    └─Conv2d: 3-1                            [2, 64, 400, 400]         (9,408)
│    │    └─BatchNorm2d: 3-2                       [2, 64, 400, 400]         (128)
│    │    └─ReLU: 3-3                              [2, 64, 400, 400]         --
│    │    └─MaxPool2d: 3-4                         [2, 64, 200, 200]         --
│    │    └─Sequential: 3-5                        [2, 256, 200, 200]        (215,808)
│    │    └─Sequential: 3-6                        [2, 512, 100, 100]        1,219,584
│    │    └─Sequential: 3-7                        [2, 1024, 50, 50]         7,098,368
│    │

In [2]:
def get_model_instance_segmentation_resnet50_fpn(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights = 'DEFAULT')

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [8]:
# 针对有网络模型，但还没有训练保存 .pth 文件的情况
import netron
import torch.onnx
from torch.autograd import Variable
from torchvision.models import resnet18  # 以 resnet18 为例

myNet = get_model_instance_segmentation_resnet50_fpn_v2(num_classes=3)  # 实例化 resnet18
x = torch.randn(2, 3, 40, 40)  # 随机生成一个输入
modelData = "./demo.pth"  # 定义模型数据保存的路径
# modelData = "./demo.onnx"  # 有人说应该是 onnx 文件，但我尝试 pth 是可以的 
torch.onnx.export(myNet, x, modelData)  # 将 pytorch 模型以 onnx 格式导出并保存
netron.start(modelData)  # 输出网络结构

#  针对已经存在网络模型 .pth 文件的情况
import netron

modelData = "./demo.pth"  # 定义模型数据保存的路径
netron.start(modelData)  # 输出网络结构


C:\Users\16477\anaconda3\envs\hy-dl\Lib\site-packages\torch\nn\functional.py:3912: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  (torch.floor((input.size(i + 2).float() * torch.tensor(scale_factors[i], dtype=torch.float32)).float()))
C:\Users\16477\anaconda3\envs\hy-dl\Lib\site-packages\torchvision\ops\boxes.py:157: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes_x = torch.min(boxes_x, torch.tensor(width, dtype=boxes.dtype, device=boxes.device))
C:\Users\16477\anaconda3\envs\hy-dl\Lib\site-packages\torchvision\ops\boxes.py:159: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), ra

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Serving './demo.pth' at http://localhost:8080
Serving './demo.pth' at http://localhost:8080


('localhost', 8080)